In [1]:
import os
import json
import shutil

import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import numpy as np
from mplfinance.original_flavor import candlestick2_ohlc
from tqdm import tqdm

# Overview of Datasets columns

In [2]:
btc_df = pd.read_csv('./../data/BTC_histData_dt1800.0s_20220825_0629.csv', sep=',')
print(btc_df.shape)
print(btc_df.columns)
btc_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './../data/BTC_histData_dt1800.0s_20220825_0629.csv'

In [31]:
print(btc_df['label'].unique())
btc_df['label'].describe()

['-' 'strong_buy' 'hold' 'sell' 'buy']


count     44665
unique        5
top           -
freq      29846
Name: label, dtype: object

In [6]:
pd.read_csv('./../data/20220828/Epi_data.csv', sep=',').columns

Index(['Unnamed: 0', 'episode', '#buy_actions', '#sell_actions', 'money',
       'fee', 'profit', 'epsilon'],
      dtype='object')

# Dataset used for training on one episode

In [9]:
act_df = pd.read_csv('./../data/20220828/action_data_e1.csv', sep=',')
print(act_df.shape)
print(act_df.columns)
act_df.head()

(6710, 13)
Index(['Unnamed: 0', 'episode', 'run', 'timestep', 'date', 'action', 'state',
       'money_free', 'money_fiktiv', 'invest', 'fee', 'reward', 'profit'],
      dtype='object')


,Unnamed: 0,episode,run,timestep,date,action,state,money_free,money_fiktiv,invest,fee,reward,profit
0,0,1,1,0,2020-03-29 04:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",200.0,200.00,0.0,0.0,0.00,0.0
1,0,1,1,1,2020-03-29 04:30:00,buy_100,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,199.80,199.8,0.2,-0.20,0.0
2,0,1,1,2,2020-03-29 05:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,199.06,199.8,0.0,-0.94,0.0
3,0,1,1,3,2020-03-29 05:30:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,197.99,199.8,0.0,-2.01,0.0
4,0,1,1,4,2020-03-29 06:00:00,hold,"[0.5, 0.9053371894725785, 0.5, 0.5, 0.46806793...",0.0,198.55,199.8,0.0,-1.45,0.0


## States

States are build of windowed values from:
 - close values of BTC
 - histogram
 - 50ema
 - rsi14
 - Money in Wallet.

In [20]:
states = list()
for ele in act_df['state']:
    states.append(json.loads(ele))
states = np.array(states)

In [24]:
states[20]

array([9.83209352e-01, 9.05337189e-01, 1.09432023e-01, 5.00000000e-01,
       4.68067937e-01, 1.04691834e-10, 8.96768986e-01, 5.13919347e-02,
       5.00000000e-01, 4.76010250e-01, 3.41704586e-15, 6.34299082e-01,
       1.61833744e-02, 4.50166003e-01, 4.36390945e-01, 9.99999975e-01,
       6.33411731e-02, 3.69794734e-02, 3.22962406e-01, 3.86211583e-01,
       1.11953595e-05, 3.08678275e-02, 2.71477320e-02, 2.54847711e-01,
       4.23724088e-01, 9.96731010e-01, 1.14207001e-02, 3.86320688e-02,
       6.37377781e-01, 4.06313170e-01, 9.56531867e-03, 1.16444019e-02,
       3.66082935e-02, 4.09218615e-01, 4.19208563e-01, 6.24450489e-20,
       1.17904044e-02, 7.56927170e-03, 5.45930762e-01, 4.11402171e-01,
       1.00000000e+00, 8.36825856e-04, 2.20130274e-02, 4.62705538e-01,
       3.45391095e-01, 2.74747063e-08, 1.04156103e-03, 1.30243059e-02,
       1.93982741e-01, 3.98769490e-01, 2.74840602e-12, 5.82990193e-04,
       5.47522884e-03, 6.75203703e-01, 3.73627613e-01, 9.99999999e-01,
      

## Actions

In [26]:
act_df['action'].unique()

array(['hold', 'buy_100', 'sell', 'buy_50'], dtype=object)

In [27]:
act_df['action'].describe()

count     6710
unique       4
top       hold
freq      6425
Name: action, dtype: object